<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/13_in_context_learning_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In-Context Learning

## 準備

In [ ]:
!pip install transformers datasets accelerate auto-gptq

**ここでランタイムを再起動する。**

## データセット
* 今回は、やさしい日本語への書き換えのデータセットを使う。
 * https://huggingface.co/datasets/snow_simplified_japanese_corpus

In [ ]:
import os
import torch
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.manual_seed(0)

In [ ]:
dataset = load_dataset("snow_simplified_japanese_corpus")

In [ ]:
dataset

In [ ]:
new_dataset = dataset.filter(
    lambda example: example["original_ja"] != example["simplified_ja"]
)

In [ ]:
new_dataset.shape

In [ ]:
dataset = new_dataset

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

In [ ]:
dataset["train"][0]

In [ ]:
dataset["test"][0]

## LLM


* 今回は、ELYZA-japanese-Llama-2-7b-instructを使う。
 * https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-instruct
* だが、Google Colab無料版では、この元のモデルは大きすぎて使えない・・・。
 * 20GB以上のメモリがあるGPUなら使える。
* そこで、量子化された下記のモデルを代わりに使う。
 * https://huggingface.co/mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k

In [ ]:
from auto_gptq import AutoGPTQForCausalLM

model_name = "mmnga/ELYZA-japanese-Llama-2-7b-fast-instruct-GPTQ-calib-ja-2k"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoGPTQForCausalLM.from_quantized(
    model_name,
    use_safetensors=True,
    device="cuda:0",
    )
model.eval()

## プロンプト
* LLMがうまく感情分析をしてくれそうなプロンプトを考える。
 * 下はあくまで一つの例。

### プロンプト作成用のヘルパ関数

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]答え："
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは優秀な日本人のアシスタントです。"

def make_prompt(sample):
  text = "次の文を、小学生でも分かるようにやさしく書き換えてください。\n"
  text += sample["original_ja"]
  prompt = "{b_inst} {system}{prompt} {e_inst} ".format(
      b_inst=B_INST,
      system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
      prompt=text,
      e_inst=E_INST,
      )
  sample["prompt"] = prompt
  return sample

* 元のテキストをプロンプトに一括変換する。

In [ ]:
dataset = dataset.map(make_prompt)

In [ ]:
dataset["train"][0]

In [ ]:
dataset["test"][0]

## やさしい日本語への書き換え

In [ ]:
for i in range(100):
  print(f'[{i+1}]' + '-'*80)
  sample = dataset["train"][i]
  original = sample["original_ja"]
  ground_truth = sample["simplified_ja"]
  prompt = sample["prompt"]
  with torch.no_grad():
    token_ids = tokenizer.encode(prompt, return_tensors="pt")
    output_ids = model.generate(
        input_ids=token_ids.to(model.device),
        max_new_tokens=50,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)
  prediction = output.split('\n')[0]
  print(f"original:\t\t{original}")
  print(f"prediction:\t{prediction}")
  print(f"ground truth:\t{ground_truth}")

In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = "あなたは、やさしい文を書くのが得意な日本人のアシスタントです。"

def make_prompt(sample):
  n_examples = 20
  text = "むずかしい文を、やさしくするには、"
  text += "どう書き換えればいいですか。\n"
  while n_examples > 0:
    index = torch.randint(0, len(dataset["train"]), (1,)).item()
    example = dataset["train"][index]
    if example["original_ja"] == sample["original_ja"]:
      continue
    text += f"例えば、\n「{example['original_ja']}」\nという文は、\n"
    text += f"「{example['simplified_ja']}」\nと書き換えればいいです。\n"
    n_examples -= 1
  text += "\nでは、あなたに質問です。\n"
  text += f"例えば、\n「{sample['original_ja']}」\nという文は、\n"
  text += "どう書き換えればいいですか。\n"
  prompt = "{b_inst} {system}{prompt} {e_inst} ".format(
      b_inst=B_INST,
      system=f"{B_SYS}{DEFAULT_SYSTEM_PROMPT}{E_SYS}",
      prompt=text,
      e_inst=E_INST,
      )
  sample["prompt"] = prompt
  return sample

In [ ]:
for i in range(100):
  print(f'[{i+1}]' + '-'*80)
  sample = dataset["train"][i]
  sample = make_prompt(sample)
  original = sample["original_ja"]
  ground_truth = sample["simplified_ja"]
  prompt = sample["prompt"]
  with torch.no_grad():
    token_ids = tokenizer.encode(prompt, return_tensors="pt")
    output_ids = model.generate(
        input_ids=token_ids.to(model.device),
        max_new_tokens=50,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
  output = tokenizer.decode(output_ids.tolist()[0][token_ids.size(1):], skip_special_tokens=True)
  prediction = output
  print(f"original:\t\t{original}")
  print(f"prediction:\t{prediction}")
  print(f"ground truth:\t{ground_truth}")

# 本日の課題
* もっとうまくLLMに書き換えをさせるプロンプトを考えてみよう。